# SFR Backtesting - Simple Analysis

This notebook provides a clean, simple interface for running SFR (Synthetic Free Risk) arbitrage backtests.

## Quick Start Guide

1. **Run Setup Cells** (Cells 1-2): Import libraries and configure database  
2. **Quick Test** (Cells 3-4): Run a 30-day SPY backtest to verify everything works  
3. **Custom Analysis** (Cells 5-6): Configure and run your own backtest  
4. **Results** (Cells 7-8): View detailed results and basic charts

## Prerequisites

- Database must be running (see `/backtesting/infra/database/` for setup)
- Python dependencies installed from `/backtesting/requirements.txt`

---

## 1. Setup & Imports

Import required libraries and configure Python path for backtesting components.

In [1]:
# Setup Python path and imports
import sys
import os
from pathlib import Path
from datetime import date, timedelta
import asyncio
import asyncpg
import pandas as pd

# Add project root to path
project_root = Path.cwd().parent.parent.parent
sys.path.insert(0, str(project_root))

# Import SFR backtesting components
from backtesting import SFRBacktestEngine, SFRBacktestConfig, SlippageModel, VixRegime
from backtesting.infra.data_collection.config.config import DatabaseConfig

print("✅ All imports successful")
print(f"📁 Project root: {project_root}")
print(f"🗓️ Today's date: {date.today()}")

✅ All imports successful
📁 Project root: /Users/ilpinto/dev/AlchimistProject/alc-lite
🗓️ Today's date: 2025-08-11


## 2. Database Configuration

Initialize database connection for backtesting.

In [2]:
# Initialize database configuration and connection
db_config = DatabaseConfig()
db_pool = None

async def setup_database():
    """Setup database connection pool"""
    global db_pool
    try:
        db_pool = await asyncpg.create_pool(
            db_config.connection_string,
            min_size=2,
            max_size=10
        )
        print("✅ Database connection established")
        print(f"🔗 Connection string: {db_config.host}:{db_config.port}/{db_config.database}")
        return True
    except Exception as e:
        print(f"❌ Database connection failed: {e}")
        print("💡 Make sure the database is running (see /backtesting/infra/database/)")
        return False

# Setup database connection
db_connected = await setup_database()

✅ Database connection established
🔗 Connection string: localhost:5433/options_arbitrage


## 3. Quick Test - 30-Day SPY Backtest

Run a simple 30-day backtest on SPY to verify everything is working correctly.

In [3]:
# FOCUSED TEST: SPY Conversion Opportunities on 2025-08-08
if not db_connected:
    print("⚠️ Database not connected - skipping test")
else:
    # Create configuration optimized for detecting engineered opportunities
    focused_config = SFRBacktestConfig(
        profit_target=0.01,      # Very low 0.01% profit target to catch opportunities
        cost_limit=2000.0,       # High cost limit to avoid filtering  
        slippage_model=SlippageModel.NONE,  # No slippage for pure detection
        commission_per_contract=0.00,       # No commission for pure detection
        volume_limit=1,          # Very low volume requirement
        max_bid_ask_spread_call=50.0,  # Wide spread tolerance
        max_bid_ask_spread_put=50.0,   # Wide spread tolerance  
        expiry_min_days=1,       # Accept very short expiries
        expiry_max_days=60       # Accept longer expiries
    )
    
    # Focused test period: Single day with the engineered data (need end > start)
    test_date = date(2025, 8, 8)  # Day with engineered opportunities
    test_end_date = date(2025, 8, 9)  # Engine requires end_date > start_date
    test_symbols = ['SPY']
    
    print("🎯 FOCUSED TEST: SPY Conversion Detection")
    print("=" * 50)
    print(f"   • Symbol: {test_symbols[0]}")
    print(f"   • Test Date: {test_date} to {test_end_date}")
    print(f"   • Engineered Opportunities Expected:")
    print(f"     - 584C/582P conversion (call_strike > put_strike ✅)")
    print(f"     - 586C/580P conversion (call_strike > put_strike ✅)")
    print(f"   • Profit Target: {focused_config.profit_target}% (very low)")
    print(f"   • Cost Limit: ${focused_config.cost_limit} (very high)")
    print(f"   • No slippage or commissions for pure detection")
    
    # Initialize focused test engine
    focused_engine = SFRBacktestEngine(db_pool, focused_config)
    print("✅ Focused test engine ready")
    
    # Verify we have SPY data for this date
    print("\n🔍 Data Verification:")
    try:
        async with db_pool.acquire() as conn:
            # Check SPY underlying ID
            spy_id = await conn.fetchval(
                "SELECT id FROM underlying_securities WHERE symbol = 'SPY'"
            )
            print(f"   • SPY underlying ID: {spy_id}")
            
            # Check stock data for test date
            stock_data = await conn.fetchrow(
                """SELECT price, volume, tick_type 
                   FROM stock_data_ticks 
                   WHERE underlying_id = $1 AND DATE(time) = $2 
                   ORDER BY time DESC LIMIT 1""", 
                spy_id, test_date
            )
            if stock_data:
                print(f"   • Stock price: ${stock_data['price']}")
                print(f"   • Volume: {stock_data['volume']:,}")  
                print(f"   • Tick type: {stock_data['tick_type']}")
            else:
                print("   ❌ No stock data found for test date")
                
            # Check option contracts for engineered expiries
            option_count = await conn.fetchval(
                """SELECT COUNT(*) FROM option_chains 
                   WHERE underlying_id = $1 
                   AND expiration_date IN ('2025-08-29', '2025-09-05', '2025-09-12', '2025-09-19')""",
                spy_id
            )
            print(f"   • Option contracts for target expiries: {option_count}")
            
            # Check for our specific strikes
            strike_check = await conn.fetch(
                """SELECT expiration_date, strike_price, option_type, COUNT(*) as count
                   FROM option_chains 
                   WHERE underlying_id = $1 
                   AND strike_price IN (580, 582, 584, 586)
                   AND expiration_date IN ('2025-08-29', '2025-09-05', '2025-09-12', '2025-09-19')
                   GROUP BY expiration_date, strike_price, option_type
                   ORDER BY expiration_date, strike_price, option_type""",
                spy_id
            )
            
            if strike_check:
                print(f"   • Target strikes found: {len(strike_check)} strike/expiry combinations")
                # Show first few examples
                for i, row in enumerate(strike_check[:6]):
                    print(f"     - {row['expiration_date']} {row['strike_price']}{row['option_type']}: {row['count']} contracts")
                if len(strike_check) > 6:
                    print(f"     - ... and {len(strike_check) - 6} more combinations")
            else:
                print("   ❌ No target strikes found")
                
    except Exception as e:
        print(f"   ❌ Data verification failed: {e}")
        
    print("\n🚀 Ready to run focused test...")

🎯 FOCUSED TEST: SPY Conversion Detection
   • Symbol: SPY
   • Test Date: 2025-08-08 to 2025-08-09
   • Engineered Opportunities Expected:
     - 584C/582P conversion (call_strike > put_strike ✅)
     - 586C/580P conversion (call_strike > put_strike ✅)
   • Profit Target: 0.01% (very low)
   • Cost Limit: $2000.0 (very high)
   • No slippage or commissions for pure detection
✅ Focused test engine ready

🔍 Data Verification:
   • SPY underlying ID: 1
   • Stock price: $581.0000
   • Volume: 56,206,163
   • Tick type: HISTORICAL
   • Option contracts for target expiries: 148
   • Target strikes found: 32 strike/expiry combinations
     - 2025-08-29 580.00C: 1 contracts
     - 2025-08-29 580.00P: 1 contracts
     - 2025-08-29 582.00C: 1 contracts
     - 2025-08-29 582.00P: 1 contracts
     - 2025-08-29 584.00C: 1 contracts
     - 2025-08-29 584.00P: 1 contracts
     - ... and 26 more combinations

🚀 Ready to run focused test...


In [4]:
# Run the focused SPY conversion test
if 'focused_engine' in locals():
    try:
        print("🚀 Running focused SPY conversion test...")
        print(f"   Testing date range: {test_date} to {test_end_date}")
        print("   Optimized for maximum opportunity detection...")
        
        # Run the focused backtest
        focused_results = await focused_engine.run_backtest(
            start_date=test_date,
            end_date=test_end_date,  # Single day range
            symbols=test_symbols
        )
        
        print("✅ Focused test completed successfully!")
        
        # Detailed analysis of results
        print("\n📊 OPPORTUNITY DETECTION RESULTS:")
        print("=" * 50)
        
        opportunities = focused_results.get('opportunities', {})
        total_found = opportunities.get('total_found', 0)
        
        print(f"   • Total Opportunities Found: {total_found}")
        
        if total_found > 0:
            print("   🎉 SUCCESS: SFR engine detected engineered opportunities!")
            
            # Quality breakdown
            by_quality = opportunities.get('by_quality', {})
            if by_quality:
                print("   • Quality Distribution:")
                for quality, count in by_quality.items():
                    print(f"     - {quality.title()}: {count}")
            
            # Symbol breakdown
            by_symbol = opportunities.get('by_symbol', {})
            if by_symbol:
                print("   • By Symbol:")
                for symbol, count in by_symbol.items():
                    print(f"     - {symbol}: {count} opportunities")
                    
        else:
            print("   ❌ NO OPPORTUNITIES DETECTED")
            print("   This suggests either:")
            print("     1. Data not properly engineered") 
            print("     2. SFR detection logic needs adjustment")
            print("     3. Market data missing for test date")
            
        # Check trades
        trades = focused_results.get('trades', {})
        total_trades = trades.get('total_simulated', 0)
        print(f"\n💰 TRADE SIMULATION:")
        print(f"   • Trades Simulated: {total_trades}")
        
        if total_trades > 0:
            # Safe handling of success_rate - could be string or float
            success_rate = trades.get('success_rate', 0)
            try:
                # Try to convert to float if it's a string
                success_rate_float = float(success_rate) if isinstance(success_rate, str) else success_rate
                print(f"   • Success Rate: {success_rate_float:.1f}%")
            except (ValueError, TypeError):
                print(f"   • Success Rate: {success_rate}")
            
            profitability = focused_results.get('profitability', {})
            net_profit = profitability.get('total_net_profit', 0)
            print(f"   • Net Profit: ${net_profit:.2f}")
            
            roi_metrics = focused_results.get('roi_metrics', {})
            avg_roi = roi_metrics.get('avg_min_roi', 0)
            best_roi = roi_metrics.get('best_min_roi', 0)
            
            # Safe handling of ROI values
            try:
                avg_roi_float = float(avg_roi) if isinstance(avg_roi, str) else avg_roi
                best_roi_float = float(best_roi) if isinstance(best_roi, str) else best_roi
                print(f"   • Average ROI: {avg_roi_float:.2f}%")
                print(f"   • Best ROI: {best_roi_float:.2f}%")
            except (ValueError, TypeError):
                print(f"   • Average ROI: {avg_roi}")
                print(f"   • Best ROI: {best_roi}")
            
        # Store results for further analysis
        focused_test_results = focused_results
        
        print(f"\n📋 Test Summary:")
        print(f"   • Backtest Run ID: {focused_results.get('backtest_run_id', 'N/A')}")
        print(f"   • Test Period: {test_date} to {test_end_date}")
        print(f"   • Symbols: {', '.join(test_symbols)}")
        print(f"   • Configuration: Low profit target, no costs")
        
        if total_found > 0:
            print("   ✅ CONVERSION OPPORTUNITIES SUCCESSFULLY DETECTED!")
        else:
            print("   ⚠️  No opportunities detected - investigate data/logic")
            
    except Exception as e:
        print(f"❌ Focused test failed: {e}")
        print(f"   Error type: {type(e).__name__}")
        print("💡 Check database connection and data integrity")
        
        # Print more detailed error info for debugging
        import traceback
        print(f"\n🔍 Detailed error:")
        print(traceback.format_exc())
        
else:
    print("⚠️ Focused engine not initialized - run previous cell first")

🚀 Running focused SPY conversion test...
   Testing date range: 2025-08-08 to 2025-08-09
   Optimized for maximum opportunity detection...
✅ Focused test completed successfully!

📊 OPPORTUNITY DETECTION RESULTS:
   • Total Opportunities Found: 32
   🎉 SUCCESS: SFR engine detected engineered opportunities!
   • Quality Distribution:
     - Fair: 32

💰 TRADE SIMULATION:
   • Trades Simulated: 2
   • Success Rate: 100.00%
   • Net Profit: $1.12
   • Average ROI: 0.09%
   • Best ROI: 0.17%

📋 Test Summary:
   • Backtest Run ID: 61
   • Test Period: 2025-08-08 to 2025-08-09
   • Symbols: SPY
   • Configuration: Low profit target, no costs
   ✅ CONVERSION OPPORTUNITIES SUCCESSFULLY DETECTED!


## 3. FOCUSED TEST: SPY Conversion Opportunities

Test the engineered SPY data for 2025-08-08 to verify conversion detection.

In [5]:
# Quick test configuration
if not db_connected:
    print("⚠️ Database not connected - skipping test")
else:
    # Create basic configuration for quick test - FIXED: Use parameters that actually work
    quick_config = SFRBacktestConfig(
        profit_target=0.01,       # 0.01% profit target (same as working focused test)
        cost_limit=2000.0,        # Higher cost limit to avoid filtering  
        slippage_model=SlippageModel.NONE,  # No slippage for demonstration
        commission_per_contract=0.00,       # No commission for demonstration
        volume_limit=1,           # Very low volume requirement (KEY FIX)
        max_bid_ask_spread_call=50.0,  # Wide spread tolerance (KEY FIX)
        max_bid_ask_spread_put=50.0,   # Wide spread tolerance (KEY FIX)
        expiry_min_days=1,        # Accept very short expiries (KEY FIX)
        expiry_max_days=60        # Accept longer expiries (KEY FIX)
    )
    
    # Test period: last 180 days (6 months)
    end_date = date.today()
    start_date = end_date - timedelta(days=180)
    test_symbols = ['SPY']  # Just SPY for quick test
    
    print("🧪 Quick Test Configuration (FIXED):")
    print(f"   • Symbol: {test_symbols[0]}")
    print(f"   • Period: {start_date} to {end_date}")
    print(f"   • Profit Target: {quick_config.profit_target}% (same as working focused test)")
    print(f"   • Cost Limit: ${quick_config.cost_limit}")
    print(f"   • Volume Limit: {quick_config.volume_limit} (KEY FIX - was default 100)")
    print(f"   • Max Call/Put Spreads: ${quick_config.max_bid_ask_spread_call} (KEY FIX - was default 20)")
    print(f"   • Expiry Range: {quick_config.expiry_min_days}-{quick_config.expiry_max_days} days (KEY FIX)")
    print(f"   • Slippage: {quick_config.slippage_model.value} (for demonstration)")
    print(f"   • Commission: ${quick_config.commission_per_contract}/contract (for demonstration)")
    
    print(f"\n💡 Configuration Notes:")
    print(f"   • Fixed volume limit from 100 to 1 to capture more opportunities")
    print(f"   • Increased bid-ask spread tolerance from 20 to 50 for realistic market data") 
    print(f"   • Extended expiry range to 1-60 days for better coverage")
    print(f"   • Should now successfully simulate trades like the focused test")
    
    # Initialize engine for quick test - FIXED: pass db_pool first
    quick_engine = SFRBacktestEngine(db_pool, quick_config)
    print("✅ Quick test engine ready")

🧪 Quick Test Configuration (FIXED):
   • Symbol: SPY
   • Period: 2025-02-12 to 2025-08-11
   • Profit Target: 0.01% (same as working focused test)
   • Cost Limit: $2000.0
   • Volume Limit: 1 (KEY FIX - was default 100)
   • Max Call/Put Spreads: $50.0 (KEY FIX - was default 20)
   • Expiry Range: 1-60 days (KEY FIX)
   • Slippage: NONE (for demonstration)
   • Commission: $0.0/contract (for demonstration)

💡 Configuration Notes:
   • Fixed volume limit from 100 to 1 to capture more opportunities
   • Increased bid-ask spread tolerance from 20 to 50 for realistic market data
   • Extended expiry range to 1-60 days for better coverage
   • Should now successfully simulate trades like the focused test
✅ Quick test engine ready


## 4. Run Quick Test

Execute the 30-day backtest and display basic results.

In [6]:
# Run the quick test
if 'quick_engine' in locals():
    try:
        print("🚀 Running 30-day SPY backtest...")
        print("   This may take 30-60 seconds...")
        
        # Run the backtest
        quick_results = await quick_engine.run_backtest(
            start_date=start_date,
            end_date=end_date, 
            symbols=test_symbols
        )
        
        print("✅ Quick test completed successfully!")
        print("\n📊 Quick Results Summary:")
        print(f"   • Opportunities Found: {quick_results.get('opportunities', {}).get('total_found', 'N/A')}")
        print(f"   • Trades Simulated: {quick_results.get('trades', {}).get('total_simulated', 'N/A')}")
        print(f"   • Success Rate: {quick_results.get('trades', {}).get('success_rate', 'N/A')}")
        print(f"   • Net Profit: ${quick_results.get('profitability', {}).get('total_net_profit', 0):.2f}")
        print(f"   • Average ROI: {quick_results.get('roi_metrics', {}).get('avg_min_roi', 'N/A')}")
        
        # Store results for later analysis
        current_results = quick_results
        
        # Display detailed trades table if trades were executed
        backtest_run_id = quick_results.get('backtest_run_id')
        trades_count = quick_results.get('trades', {}).get('total_simulated', 0)
        
        if backtest_run_id and trades_count > 0:
            print(f"\n📋 DETAILED TRADES ANALYSIS (Run ID: {backtest_run_id})")
            print("=" * 70)
            
            try:
                # Query individual trades
                async with db_pool.acquire() as conn:
                    individual_trades = await conn.fetch("""
                        SELECT 
                            SUBSTRING(t.trade_id::text, 1, 8) as trade_id,
                            DATE(t.execution_timestamp) as trade_date,
                            us.symbol,
                            o.call_strike || 'C/' || o.put_strike || 'P' as strategy,
                            ROUND(o.stock_price::numeric, 2) as stock_price,
                            ROUND(t.realized_min_profit::numeric, 2) as profit_usd,
                            ROUND(t.realized_min_roi::numeric, 4) as roi_percent,
                            t.execution_quality
                        FROM sfr_simulated_trades t
                        JOIN sfr_opportunities o ON t.opportunity_id = o.id
                        JOIN underlying_securities us ON o.underlying_id = us.id
                        WHERE t.execution_status = 'FILLED'
                            AND t.backtest_run_id = $1
                        ORDER BY t.execution_timestamp DESC
                        LIMIT 15
                    """, backtest_run_id)
                    
                    if individual_trades:
                        print("\n🔍 Recent Individual Trades:")
                        trades_df = pd.DataFrame(individual_trades, columns=[
                            'Trade ID', 'Date', 'Symbol', 'Strategy', 'Stock Price', 
                            'Profit ($)', 'ROI (%)', 'Quality'
                        ])
                        print(trades_df.to_string(index=False))
                    
                    # Query strategy performance summary
                    strategy_summary = await conn.fetch("""
                        SELECT 
                            us.symbol,
                            o.call_strike || 'C/' || o.put_strike || 'P' as strategy,
                            COUNT(*) as num_trades,
                            ROUND(AVG(t.realized_min_profit::numeric), 2) as avg_profit,
                            ROUND(SUM(t.realized_min_profit::numeric), 2) as total_profit,
                            ROUND(AVG(t.realized_min_roi::numeric), 4) as avg_roi_percent,
                            ROUND(MIN(t.realized_min_profit::numeric), 2) as min_profit,
                            ROUND(MAX(t.realized_min_profit::numeric), 2) as max_profit
                        FROM sfr_simulated_trades t
                        JOIN sfr_opportunities o ON t.opportunity_id = o.id
                        JOIN underlying_securities us ON o.underlying_id = us.id
                        WHERE t.execution_status = 'FILLED'
                            AND t.backtest_run_id = $1
                        GROUP BY us.symbol, o.call_strike, o.put_strike
                        ORDER BY total_profit DESC
                    """, backtest_run_id)
                    
                    if strategy_summary:
                        print(f"\n📈 Strategy Performance Summary:")
                        strategy_df = pd.DataFrame(strategy_summary, columns=[
                            'Symbol', 'Strategy', 'Trades', 'Avg Profit', 'Total Profit', 
                            'Avg ROI (%)', 'Min Profit', 'Max Profit'
                        ])
                        print(strategy_df.to_string(index=False))
                    
                    # Overall performance metrics
                    overall_stats = await conn.fetchrow("""
                        SELECT 
                            COUNT(*) as total_trades,
                            ROUND(SUM(t.realized_min_profit::numeric), 2) as total_profit,
                            ROUND(AVG(t.realized_min_profit::numeric), 2) as avg_profit,
                            ROUND(MIN(t.realized_min_profit::numeric), 2) as min_profit,
                            ROUND(MAX(t.realized_min_profit::numeric), 2) as max_profit,
                            ROUND(AVG(t.realized_min_roi::numeric), 4) as avg_roi,
                            COUNT(DISTINCT DATE(t.execution_timestamp)) as trading_days,
                            COUNT(DISTINCT (o.call_strike || '/' || o.put_strike)) as unique_strategies
                        FROM sfr_simulated_trades t
                        JOIN sfr_opportunities o ON t.opportunity_id = o.id
                        WHERE t.execution_status = 'FILLED'
                            AND t.backtest_run_id = $1
                    """, backtest_run_id)
                    
                    if overall_stats:
                        print(f"\n🎯 Overall Performance Metrics:")
                        print(f"   • Total Successful Trades: {overall_stats['total_trades']}")
                        print(f"   • Total Profit: ${overall_stats['total_profit']}")
                        print(f"   • Average Profit/Trade: ${overall_stats['avg_profit']}")
                        print(f"   • Best Trade: ${overall_stats['max_profit']}")
                        print(f"   • Worst Trade: ${overall_stats['min_profit']}")
                        print(f"   • Average ROI: {overall_stats['avg_roi']}%")
                        print(f"   • Active Trading Days: {overall_stats['trading_days']}")
                        print(f"   • Unique Strategies Used: {overall_stats['unique_strategies']}")
                        
            except Exception as e:
                print(f"   ⚠️ Could not load detailed trades: {e}")
                
        elif trades_count == 0:
            print(f"\n💡 No trades were simulated in this backtest.")
            print("   • Try lowering profit_target or adjusting other configuration parameters")
            print("   • Check that opportunities were found (should be > 0)")
        
    except Exception as e:
        print(f"❌ Quick test failed: {e}")
        print("💡 Check database connection and data availability")
        
else:
    print("⚠️ Quick engine not initialized - run previous cell first")

🚀 Running 30-day SPY backtest...
   This may take 30-60 seconds...
✅ Quick test completed successfully!

📊 Quick Results Summary:
   • Opportunities Found: 672
   • Trades Simulated: 60
   • Success Rate: 100.00%
   • Net Profit: $83.40
   • Average ROI: 0.23%

📋 DETAILED TRADES ANALYSIS (Run ID: 62)

🔍 Recent Individual Trades:
Trade ID       Date Symbol        Strategy Stock Price Profit ($) ROI (%)   Quality
336ab671 2025-08-07    SPY 570.00C/566.00P      581.00       0.12  0.0204 EXCELLENT
f0ce39d0 2025-08-07    SPY 570.00C/568.00P      581.00       1.00  0.1681 EXCELLENT
5eb0ec75 2025-08-06    SPY 572.00C/570.00P      583.56       1.44  0.2398 EXCELLENT
2fc9835e 2025-08-05    SPY 572.00C/570.00P      583.11       1.89  0.3152 EXCELLENT
8564f75a 2025-08-04    SPY 572.00C/570.00P      583.16       1.84  0.3076 EXCELLENT
e6184dad 2025-08-03    SPY 572.00C/568.00P      581.95       0.55  0.0925 EXCELLENT
6af8135b 2025-08-03    SPY 572.00C/570.00P      581.95       3.05  0.5112 EXCELLE

## 5. Custom Backtest Configuration

Configure your own backtest with custom parameters, symbols, and date range.

In [7]:
# Custom backtest configuration
# Modify these parameters for your analysis

# === BACKTEST PARAMETERS ===
custom_profit_target = 0.005    # Profit target percentage (0.75 = 0.75%)
custom_cost_limit = 1000.0      # Maximum cost per trade ($)
custom_slippage_model = SlippageModel.LINEAR  # Options: NONE, LINEAR, SQUARE_ROOT, IMPACT
custom_commission = 1.00       # Commission per contract ($)

# === DATE RANGE ===  
# Option 1: Use predefined periods
period_months = 6  # Last 6 months
custom_end_date = date.today()
custom_start_date = custom_end_date - timedelta(days=period_months * 30)

# Option 2: Specify exact dates (uncomment to use)
# custom_start_date = date(2023, 1, 1)  
# custom_end_date = date(2023, 12, 31)

# === SYMBOLS ===
# Popular options for SFR backtesting
custom_symbols = [
    'SPY',   # S&P 500 ETF - most liquid
    # 'QQQ',   # NASDAQ ETF - tech heavy
    # 'AAPL',  # Apple - high volume individual stock  
    # 'MSFT',  # Microsoft - stable large cap
    # 'TSLA'   # Tesla - high volatility stock
]

# Alternative symbol sets (uncomment to use)
# custom_symbols = ['SPY', 'QQQ']  # ETF only
# custom_symbols = ['AAPL', 'MSFT', 'NVDA', 'TSLA', 'META']  # Tech stocks only

# === CREATE CONFIGURATION ===
custom_config = SFRBacktestConfig(
    profit_target=custom_profit_target,
    cost_limit=custom_cost_limit,
    slippage_model=custom_slippage_model,
    commission_per_contract=custom_commission,
    # Additional optional parameters
    volume_limit=50,           # Minimum option volume
    max_bid_ask_spread_call=15.0,  # Max call spread
    max_bid_ask_spread_put=15.0,   # Max put spread
    expiry_min_days=15,        # Min days to expiration
    expiry_max_days=45         # Max days to expiration
)

print("🔧 Custom Backtest Configuration:")
print(f"   • Symbols: {', '.join(custom_symbols)}")
print(f"   • Period: {custom_start_date} to {custom_end_date} ({(custom_end_date - custom_start_date).days} days)")
print(f"   • Profit Target: {custom_config.profit_target}%")
print(f"   • Cost Limit: ${custom_config.cost_limit}")
print(f"   • Slippage Model: {custom_config.slippage_model.value}")
print(f"   • Commission: ${custom_config.commission_per_contract}/contract")

# Initialize custom engine - FIXED: pass db_pool first
if db_connected:
    custom_engine = SFRBacktestEngine(db_pool, custom_config)
    print("✅ Custom engine ready")
else:
    print("⚠️ Database not connected - cannot create engine")

🔧 Custom Backtest Configuration:
   • Symbols: SPY
   • Period: 2025-02-12 to 2025-08-11 (180 days)
   • Profit Target: 0.005%
   • Cost Limit: $1000.0
   • Slippage Model: LINEAR
   • Commission: $1.0/contract
✅ Custom engine ready


## 6. Run Custom Backtest

Execute the custom backtest with your configured parameters.

In [8]:
# Run the custom backtest
if 'custom_engine' in locals():
    try:
        print("🚀 Running custom SFR backtest...")
        print(f"   Analyzing {len(custom_symbols)} symbols over {(custom_end_date - custom_start_date).days} days")
        print("   This may take 2-5 minutes depending on the time period and symbols...")
        
        # Run the backtest
        custom_results = await custom_engine.run_backtest(
            start_date=custom_start_date,
            end_date=custom_end_date,
            symbols=custom_symbols
        )
        
        print("✅ Custom backtest completed successfully!")
        print(f"   Backtest ID: {custom_results.get('backtest_run_id', 'N/A')}")
        
        # Store results for detailed analysis
        current_results = custom_results
        
        # Display summary
        print("\n📊 Backtest Summary:")
        print(f"   • Period: {custom_results.get('period', {}).get('start_date')} to {custom_results.get('period', {}).get('end_date')}")
        print(f"   • Total Days: {custom_results.get('period', {}).get('total_days', 'N/A')}")
        print(f"   • Symbols Analyzed: {', '.join(custom_results.get('symbol_coverage', []))}")
        
        print("\n📈 Opportunity Discovery:")
        opportunities = custom_results.get('opportunities', {})
        print(f"   • Total Found: {opportunities.get('total_found', 'N/A')}")
        print(f"   • Per Day: {opportunities.get('per_day', 'N/A'):.2f}")
        
        # Quality breakdown
        by_quality = opportunities.get('by_quality', {})
        for quality, count in by_quality.items():
            print(f"   • {quality.title()}: {count}")
        
        print("\n💰 Trading Performance:")
        trades = custom_results.get('trades', {})
        profitability = custom_results.get('profitability', {})
        print(f"   • Total Trades: {trades.get('total_simulated', 'N/A')}")
        print(f"   • Success Rate: {trades.get('success_rate', 'N/A')}")
        print(f"   • Net Profit: ${profitability.get('total_net_profit', 0):.2f}")
        print(f"   • Avg Profit/Trade: ${profitability.get('avg_profit_per_trade', 0):.2f}")
        
        roi_metrics = custom_results.get('roi_metrics', {})
        print(f"   • Average ROI: {roi_metrics.get('avg_min_roi', 'N/A')}")
        print(f"   • Best ROI: {roi_metrics.get('best_min_roi', 'N/A')}")
        
        # Display detailed trades table if trades were executed
        backtest_run_id = custom_results.get('backtest_run_id')
        trades_count = custom_results.get('trades', {}).get('total_simulated', 0)
        
        if backtest_run_id and trades_count > 0:
            print(f"\n📋 DETAILED TRADES ANALYSIS (Run ID: {backtest_run_id})")
            print("=" * 70)
            
            try:
                # Query individual trades
                async with db_pool.acquire() as conn:
                    individual_trades = await conn.fetch("""
                        SELECT 
                            SUBSTRING(t.trade_id::text, 1, 8) as trade_id,
                            DATE(t.execution_timestamp) as trade_date,
                            us.symbol,
                            o.call_strike || 'C/' || o.put_strike || 'P' as strategy,
                            ROUND(o.stock_price::numeric, 2) as stock_price,
                            ROUND(t.realized_min_profit::numeric, 2) as profit_usd,
                            ROUND(t.realized_min_roi::numeric, 4) as roi_percent,
                            ROUND(t.total_commission::numeric, 2) as commission,
                            ROUND(t.total_slippage::numeric, 2) as slippage,
                            t.execution_quality
                        FROM sfr_simulated_trades t
                        JOIN sfr_opportunities o ON t.opportunity_id = o.id
                        JOIN underlying_securities us ON o.underlying_id = us.id
                        WHERE t.execution_status = 'FILLED'
                            AND t.backtest_run_id = $1
                        ORDER BY t.execution_timestamp DESC
                        LIMIT 20
                    """, backtest_run_id)
                    
                    if individual_trades:
                        print("\n🔍 Individual Trades Details:")
                        trades_df = pd.DataFrame(individual_trades, columns=[
                            'Trade ID', 'Date', 'Symbol', 'Strategy', 'Stock Price', 
                            'Profit ($)', 'ROI (%)', 'Commission', 'Slippage', 'Quality'
                        ])
                        print(trades_df.to_string(index=False))
                    
                    # Query strategy performance summary
                    strategy_summary = await conn.fetch("""
                        SELECT 
                            us.symbol,
                            o.call_strike || 'C/' || o.put_strike || 'P' as strategy,
                            COUNT(*) as num_trades,
                            ROUND(AVG(t.realized_min_profit::numeric), 2) as avg_profit,
                            ROUND(SUM(t.realized_min_profit::numeric), 2) as total_profit,
                            ROUND(AVG(t.realized_min_roi::numeric), 4) as avg_roi_percent,
                            ROUND(MIN(t.realized_min_profit::numeric), 2) as min_profit,
                            ROUND(MAX(t.realized_min_profit::numeric), 2) as max_profit,
                            ROUND(AVG(t.total_commission::numeric), 2) as avg_commission,
                            ROUND(AVG(t.total_slippage::numeric), 2) as avg_slippage
                        FROM sfr_simulated_trades t
                        JOIN sfr_opportunities o ON t.opportunity_id = o.id
                        JOIN underlying_securities us ON o.underlying_id = us.id
                        WHERE t.execution_status = 'FILLED'
                            AND t.backtest_run_id = $1
                        GROUP BY us.symbol, o.call_strike, o.put_strike
                        ORDER BY total_profit DESC
                    """, backtest_run_id)
                    
                    if strategy_summary:
                        print(f"\n📈 Strategy Performance Summary:")
                        strategy_df = pd.DataFrame(strategy_summary, columns=[
                            'Symbol', 'Strategy', 'Trades', 'Avg Profit', 'Total Profit', 
                            'Avg ROI (%)', 'Min Profit', 'Max Profit', 'Avg Commission', 'Avg Slippage'
                        ])
                        print(strategy_df.to_string(index=False))
                    
                    # Overall performance metrics with costs breakdown
                    overall_stats = await conn.fetchrow("""
                        SELECT 
                            COUNT(*) as total_trades,
                            ROUND(SUM(t.realized_min_profit::numeric), 2) as total_profit,
                            ROUND(AVG(t.realized_min_profit::numeric), 2) as avg_profit,
                            ROUND(MIN(t.realized_min_profit::numeric), 2) as min_profit,
                            ROUND(MAX(t.realized_min_profit::numeric), 2) as max_profit,
                            ROUND(AVG(t.realized_min_roi::numeric), 4) as avg_roi,
                            ROUND(SUM(t.total_commission::numeric), 2) as total_commission,
                            ROUND(SUM(t.total_slippage::numeric), 2) as total_slippage,
                            COUNT(DISTINCT DATE(t.execution_timestamp)) as trading_days,
                            COUNT(DISTINCT (o.call_strike || '/' || o.put_strike)) as unique_strategies,
                            COUNT(DISTINCT us.symbol) as symbols_traded
                        FROM sfr_simulated_trades t
                        JOIN sfr_opportunities o ON t.opportunity_id = o.id
                        JOIN underlying_securities us ON o.underlying_id = us.id
                        WHERE t.execution_status = 'FILLED'
                            AND t.backtest_run_id = $1
                    """, backtest_run_id)
                    
                    if overall_stats:
                        print(f"\n🎯 Overall Performance Metrics:")
                        print(f"   • Total Successful Trades: {overall_stats['total_trades']}")
                        print(f"   • Gross Profit: ${overall_stats['total_profit']}")
                        print(f"   • Total Commissions: ${overall_stats['total_commission']}")
                        print(f"   • Total Slippage: ${overall_stats['total_slippage']}")
                        net_after_costs = overall_stats['total_profit'] - overall_stats['total_commission'] - overall_stats['total_slippage']
                        print(f"   • Net Profit (after costs): ${net_after_costs:.2f}")
                        print(f"   • Average Profit/Trade: ${overall_stats['avg_profit']}")
                        print(f"   • Best Trade: ${overall_stats['max_profit']}")
                        print(f"   • Worst Trade: ${overall_stats['min_profit']}")
                        print(f"   • Average ROI: {overall_stats['avg_roi']}%")
                        print(f"   • Active Trading Days: {overall_stats['trading_days']}")
                        print(f"   • Symbols Traded: {overall_stats['symbols_traded']}")
                        print(f"   • Unique Strategies Used: {overall_stats['unique_strategies']}")
                        
                        # Performance analysis
                        if net_after_costs > 0:
                            print(f"   🎉 PROFITABLE: Strategy generated positive returns after all costs")
                        elif net_after_costs == 0:
                            print(f"   ➖ BREAK-EVEN: Strategy covered costs but no profit")
                        else:
                            print(f"   ⚠️  LOSS: Strategy lost money after costs (commission/slippage too high)")
                        
            except Exception as e:
                print(f"   ⚠️ Could not load detailed trades: {e}")
                
        elif trades_count == 0:
            print(f"\n💡 No trades were simulated in this backtest.")
            print("   • Try adjusting configuration parameters:")
            print("   • Lower profit_target (currently {:.3f}%)".format(custom_config.profit_target))
            print("   • Increase cost_limit (currently ${})".format(custom_config.cost_limit))
            print("   • Lower volume_limit (currently {})".format(custom_config.volume_limit))
            print("   • Increase max_bid_ask_spread tolerance")
            print("   • Extend expiry_min_days and expiry_max_days range")
        
    except Exception as e:
        print(f"❌ Custom backtest failed: {e}")
        print(f"   Error type: {type(e).__name__}")
        print("💡 Check database connection, data availability, or reduce date range")
        
else:
    print("⚠️ Custom engine not initialized - run previous cell first")

🚀 Running custom SFR backtest...
   Analyzing 1 symbols over 180 days
   This may take 2-5 minutes depending on the time period and symbols...
✅ Custom backtest completed successfully!
   Backtest ID: 63

📊 Backtest Summary:
   • Period: 2025-02-12 to 2025-08-11
   • Total Days: 180
   • Symbols Analyzed: SPY

📈 Opportunity Discovery:
   • Total Found: 672
   • Per Day: 3.73
   • Unknown: 512
   • Fair: 160

💰 Trading Performance:
   • Total Trades: 3
   • Success Rate: 100.00%
   • Net Profit: $-8.65
   • Avg Profit/Trade: $-2.88
   • Average ROI: -0.48%
   • Best ROI: -0.46%

📋 DETAILED TRADES ANALYSIS (Run ID: 63)

🔍 Individual Trades Details:
Trade ID       Date Symbol        Strategy Stock Price Profit ($) ROI (%) Commission Slippage   Quality
874e468c 2025-08-07    SPY 562.00C/560.00P      581.00      -2.83 -0.4703       3.00     0.12 EXCELLENT
70af24f4 2025-08-07    SPY 564.00C/562.00P      581.00      -3.04 -0.5069       3.00     0.12 EXCELLENT
57a17723 2025-08-07    SPY 568.00

## 7. Detailed Results Analysis

View comprehensive performance metrics, risk analysis, and configuration details.

In [9]:
# Detailed results analysis
if 'current_results' in locals() and current_results:
    print("📋 DETAILED BACKTEST RESULTS")
    print("=" * 50)
    
    # Configuration Details
    print("\n🔧 CONFIGURATION:")
    config = current_results.get('config', {})
    print(f"   Profit Target: {config.get('profit_target', 'N/A')}%")
    print(f"   Cost Limit: ${config.get('cost_limit', 'N/A')}")
    print(f"   Slippage Model: {config.get('slippage_model', 'N/A')}")
    print(f"   Commission/Contract: ${config.get('commission_per_contract', 'N/A')}")
    print(f"   Volume Limit: {config.get('volume_limit', 'N/A')}")
    print(f"   Expiry Range: {config.get('expiry_min_days', 'N/A')}-{config.get('expiry_max_days', 'N/A')} days")
    
    # Market Analysis
    print("\n📊 MARKET ANALYSIS:")
    period = current_results.get('period', {})
    print(f"   Start Date: {period.get('start_date', 'N/A')}")
    print(f"   End Date: {period.get('end_date', 'N/A')}")
    print(f"   Total Trading Days: {period.get('total_days', 'N/A')}")
    print(f"   Symbols Coverage: {', '.join(current_results.get('symbol_coverage', []))}")
    
    # Financial Performance
    print("\n💰 FINANCIAL PERFORMANCE:")
    profitability = current_results.get('profitability', {})
    print(f"   Gross Profit: ${profitability.get('total_gross_profit', 0):.2f}")
    print(f"   Total Commissions: ${profitability.get('total_commissions', 0):.2f}")
    print(f"   Total Slippage Cost: ${profitability.get('total_slippage_cost', 0):.2f}")
    print(f"   Net Profit: ${profitability.get('total_net_profit', 0):.2f}")
    print(f"   Average Profit/Trade: ${profitability.get('avg_profit_per_trade', 0):.2f}")
    
    # Performance Summary
    print("\n🎯 SUMMARY:")
    net_profit = profitability.get('total_net_profit', 0)
    trades = current_results.get('trades', {})
    
    if net_profit > 0:
        print("   ✅ PROFITABLE STRATEGY")
    elif net_profit == 0:
        print("   ➖ BREAK-EVEN STRATEGY")  
    else:
        print("   ❌ UNPROFITABLE STRATEGY")
        
    opportunities = current_results.get('opportunities', {})
    print(f"   💡 Trade Frequency: {opportunities.get('per_day', 0):.2f} opportunities/day")
    
else:
    print("⚠️ No results available - run a backtest first")

📋 DETAILED BACKTEST RESULTS

🔧 CONFIGURATION:
   Profit Target: 0.005%
   Cost Limit: $1000.0
   Slippage Model: LINEAR
   Commission/Contract: $1.0
   Volume Limit: N/A
   Expiry Range: N/A-N/A days

📊 MARKET ANALYSIS:
   Start Date: 2025-02-12
   End Date: 2025-08-11
   Total Trading Days: 180
   Symbols Coverage: SPY

💰 FINANCIAL PERFORMANCE:
   Gross Profit: $0.35
   Total Commissions: $9.00
   Total Slippage Cost: $213.45
   Net Profit: $-8.65
   Average Profit/Trade: $-2.88

🎯 SUMMARY:
   ❌ UNPROFITABLE STRATEGY
   💡 Trade Frequency: 3.73 opportunities/day


## 8. Cleanup & Next Steps

Clean up resources and suggestions for further analysis.

In [10]:
# Cleanup resources
if 'db_pool' in locals() and db_pool:
    try:
        await db_pool.close()
        print("✅ Database connection closed")
    except Exception as e:
        print(f"⚠️ Database cleanup warning: {e}")

print("🎉 SFR Backtesting Analysis Complete!")
print("\n📚 Next Steps & Advanced Usage:")
print("   • Export results to JSON for further analysis")
print("   • Run longer time periods (1-5 years) for comprehensive analysis") 
print("   • Test more symbols (individual stocks vs ETFs)")
print("   • Experiment with different slippage models")
print("   • Analyze seasonal patterns and market regime changes")

print("\n🔧 Additional Tools Available:")
print("   • Command-line interface: /backtesting/examples/run_sfr_backtest.py")
print("   • Configuration presets: Conservative, Aggressive, VIX-filtered")
print("   • Database tools: /backtesting/infra/database/")
print("   • Data collection: /backtesting/infra/data_collection/")

if 'current_results' in locals() and current_results:
    print(f"\n📋 Last Analysis Summary:")
    print(f"   • Backtest ID: {current_results.get('backtest_run_id', 'N/A')}")
    print(f"   • Net Profit: ${current_results.get('profitability', {}).get('total_net_profit', 0):.2f}")
    print(f"   • Success Rate: {current_results.get('trades', {}).get('success_rate', 'N/A')}")
    print(f"   • Total Opportunities: {current_results.get('opportunities', {}).get('total_found', 'N/A')}")

print("\n🚀 Ready for production SFR backtesting!")

✅ Database connection closed
🎉 SFR Backtesting Analysis Complete!

📚 Next Steps & Advanced Usage:
   • Export results to JSON for further analysis
   • Run longer time periods (1-5 years) for comprehensive analysis
   • Test more symbols (individual stocks vs ETFs)
   • Experiment with different slippage models
   • Analyze seasonal patterns and market regime changes

🔧 Additional Tools Available:
   • Command-line interface: /backtesting/examples/run_sfr_backtest.py
   • Configuration presets: Conservative, Aggressive, VIX-filtered
   • Database tools: /backtesting/infra/database/
   • Data collection: /backtesting/infra/data_collection/

📋 Last Analysis Summary:
   • Backtest ID: 63
   • Net Profit: $-8.65
   • Success Rate: 100.00%
   • Total Opportunities: 672

🚀 Ready for production SFR backtesting!
